<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/DeployToProduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📥 Step 1: Upload the `.env` File

This notebook requires a `.env` file containing your API keys and credentials. Create a file with the following environment variables:

| Variable Name | Purpose |
|---|---|
| `OPENAI_API_KEY` | Access to OpenAI API |
| `AWS_ACCESS_KEY_ID` | AWS access key |
| `AWS_SECRET_ACCESS_KEY` | AWS secret access key |
| `AWS_DEFAULT_REGION` | AWS REGION |
| `S3_BUCKET_NAME` | Your S3 bucket name |
| `IMAGE_FOLDER_NAME` | Image directory |
| `IMAGES_DIR` | Directory path for images |
| `ELEPHANT_PRIVATE_KEY` | Elephant wallet private key |
| `PINATA_JWT` | Pinata authentication token |
|`NETLIFY_SITE_ID` | Netlify project ID |
|`NETLIFY_TOKEN` | Netlify token |

### To upload:
1. Click the **folder icon** 📂 in the left sidebar
2. Click the **"Upload"** button
3. Select your `.env` file

### Example `.env` file:
```env
OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXX
AWS_ACCESS_KEY_ID=XXXXXX
AWS_SECRET_ACCESS_KEY=XXXXXX
S3_BUCKET_NAME=your-s3-bucket-name-here
IMAGES_DIR=images
IMAGE_FOLDER_NAME=images
ELEPHANT_PRIVATE_KEY=xxxxx
PINATA_JWT=xxxxx
NETLIFY_SITE_ID=xxxxx
NETLIFY_TOKEN=xxxxxxxx

```

> ⚠️ **Security Note:** Never commit your `.env` file to version control or share it publicly.


## Step 2: Upload upload-results.csv

Upload the upload-results.csv file from any step that you want to push the HTML to production.

👉 Make sure to upload it to the content/ folder. This is required for the production pipeline to pick it up properly.


In [ ]:
!ls -la /content/upload-results.csv

## Step 3 Set up to be able to deploy to production


In [ ]:
%%bash

# Elephant Fact Sheet Template Installer (Silent)
set -e  # Exit on any error

INSTALL_DIR="${HOME}/.elephant-fact-sheet"
BIN_DIR="${HOME}/.local/bin"
STATIC_TARGET_DIR="$BIN_DIR/templates/assets/static"
TEST="/content/templates/assets/static"

# Check Node.js version
if ! command -v node &> /dev/null; then
    exit 1
fi

NODE_VERSION=$(node -v | cut -d'v' -f2 | cut -d'.' -f1)
if [ "$NODE_VERSION" -lt 18 ]; then
    exit 1
fi

# Check npm
if ! command -v npm &> /dev/null; then
    exit 1
fi

# Clean up any existing installation
rm -rf "$INSTALL_DIR"

# Clone repo silently
git clone --quiet https://github.com/elephant-xyz/fact-sheet-template.git "$INSTALL_DIR"

cd "$INSTALL_DIR"

# Silent npm install and build
npm install --silent --no-audit --no-fund > /dev/null 2>&1
npm run build --silent > /dev/null 2>&1

# Setup directories and copy assets
mkdir -p "$BIN_DIR" "$STATIC_TARGET_DIR" "$TEST"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$STATIC_TARGET_DIR"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$TEST"

# Link and make executable
ln -sf "$INSTALL_DIR/bin/fact-sheet.js" "$BIN_DIR/fact-sheet"
chmod +x "$INSTALL_DIR/bin/fact-sheet.js"

## Step 2